In [30]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten


importing the MNIST dataset

In [24]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

#make sure that the pixel values are between 0 and 1 
 #(Since the pixels can be RGB, and RGB goes from 0 to 255)
x_train, x_test = x_train / 255.0, x_test / 255.0

assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

print(x_train.shape, y_train)

(60000, 28, 28) [5 0 4 ... 5 6 8]


## step 1 specify the model.

**note the ouput layer has more than 2 units.

In [39]:
model=Sequential([
    #this takes the inputs, which are 28x28 images, and forms 2D single line vectors
    Flatten(input_shape=(28,28)),

#the number of neurons, just always try 256, 1024 etc etc until 4096
    #by James's experience, this is the best
    Dense(units=250, activation='relu'),
    Dense(units=250, activation='relu'),
    # Dense(units=10, activation='softmax')
    Dense(units=10, activation='linear')
])

# step 2: specify the loss and cost

**the cost function is also different, since we are not using binary crossentropy (we don't have only 2 possible outputs, instead we need the piecewise function with the logs for EVERY activation possible)

In [45]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
# model.compile(loss=SparseCategoricalCrossentropy())

#this line, from_logits=True, means that the output layer is a linear layer, and the activation function is the softmax function.
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    #the metrics=["accuracy"] is just so that when I train, I can see the accuracy

In [46]:
model.fit(x_train,y_train, epochs=10, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 9s 5ms/step - loss: 3.9928e-07 - accuracy: 1.0000 - val_loss: 0.0181 - val_accuracy: 0.9973
Epoch 2/10
1688/1688 [==============================] - 8s 5ms/step - loss: 3.7167e-07 - accuracy: 1.0000 - val_loss: 0.0182 - val_accuracy: 0.9973
Epoch 3/10
1688/1688 [==============================] - 9s 5ms/step - loss: 3.4820e-07 - accuracy: 1.0000 - val_loss: 0.0183 - val_accuracy: 0.9973
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 3.3045e-07 - accuracy: 1.0000 - val_loss: 0.0182 - val_accuracy: 0.9973
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 3.1368e-07 - accuracy: 1.0000 - val_loss: 0.0183 - val_accuracy: 0.9973
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 2.9925e-07 - accuracy: 1.0000 - val_loss: 0.0183 - val_accuracy: 0.9973
Epoch 7/10
1688/1688 [==============================] - 9s 6ms/step - loss: 2.8626e-07 - accuracy: 1.0000 - val

In [44]:
model.evaluate(x_test,y_test)
model.summary()
    

313/313 [==============================] - 1s 2ms/step - loss: 0.1379 - accuracy: 0.9852


[0.13794299960136414, 0.9851999878883362]

RAndom stuff that doesn't relate to the machine learning    

In [47]:
import matplotlib.pyplot as plt
import numpy as np

def plot_images(images,labels,img_shape,layout,cmap="gray"):
  if len(labels) == 0:
    labels = [""] * len(images)
  area = layout[0] * layout[1]
  rows, cols = layout
  fig, ax = plt.subplots(rows,cols,figsize=(rows*2,cols*2))
  fig.tight_layout()
  # fig.tight_layout(h_pad=0)
  for i, subplot in enumerate(ax.flatten()):
    if i >= len(images):
      break
    img = np.reshape(images[i],img_shape)
    subplot.imshow(img,cmap=cmap)
    subplot.set_title(labels[i])
    subplot.set_xticks([])
    subplot.set_yticks([])
  plt.show()

# Visualizing our models
Let's see how well our model performs by drawing digits ourselves! We will use `Gradio`, a library that allows you to make interactive visualizations right inside Colab itself!

In [48]:
#!pip install gradio
import gradio as gr
def classify(image):
  # describe(image,"image")
  prediction = model.predict(image.reshape(-1,28,28) / 255).tolist()[0]
  return {str(i): prediction[i] for i in range(10)}
  
sketchpad = gr.Image(
                  image_mode='L', 
                  source='canvas', 
                  shape=(28, 28), 
                  invert_colors=True, 
                  tool= 'select')
label = gr.Label(num_top_classes=3)
interface = gr.Interface(classify, sketchpad, label)
interface.launch(share=True,debug=False);

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://9f66c396aa83eb0905.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 26ms/step
